In [1]:
%matplotlib inline


# Image Classification


In [2]:
import numpy as np

import sklearn.model_selection

import torchvision.datasets

from autoPyTorch.pipeline.image_classification import ImageClassificationPipeline

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch

In [3]:
generator1 = torch.Generator().manual_seed(42)

In [4]:
# Define the dataset path and transformation
dataset_path = "./Project 3/apple_resized_224/Train"
transform_img_normal = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = ImageFolder(dataset_path, transform=transform_img_normal)

# split in train and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size],generator=generator1)


In [5]:
# Extract properties from the subset object
dataset = train_dataset.dataset
indices = train_dataset.indices

# Create a dictionary with the extracted properties
dataset_properties = {
    'dataset': dataset,
    'indices': indices
}

# Extract properties from the subset object
testdataset = test_dataset.dataset
testindices = test_dataset.indices

# Create a dictionary with the extracted properties
testdataset_properties = {
    'dataset': testdataset,
    'indices': testindices
}

In [6]:
data = dataset_properties

pipeline = ImageClassificationPipeline(dataset_properties=data)
print(pipeline)



________________________________________
	ImageClassificationPipeline
________________________________________
0-) normalizer: 
	ImageNormalizer

1-) preprocessing: 
	EarlyPreprocessing

2-) image_augmenter: 
	ImageAugmenter

________________________________________


In [7]:
# Calculate mean and standard deviation
mean = torch.stack([torch.mean(img, dim=0) for img, _ in train_dataset])
std = torch.stack([torch.std(img, dim=0) for img, _ in train_dataset])

In [8]:
# Configuration space
pipeline_cs = pipeline.get_hyperparameter_search_space()
print("Pipeline CS:\n", '_' * 40, f"\n{pipeline_cs}")
config = pipeline_cs.sample_configuration()
print("Pipeline Random Config:\n", '_' * 40, f"\n{config}")
pipeline.set_hyperparameters(config)


# pipeline.fit(X=dict(X_train=data,
#                     is_small_preprocess=True,
#                     dataset_properties=dict(mean=mean.numpy(),
#                                             std=std.numpy(),
#                                             num_classes=4,
#                                             num_features=dataset[0][0].shape[0] * dataset[0][0].shape[1],
#                                             image_height=dataset[0][0].shape[1],
#                                             image_width=dataset[0][0].shape[2],
#                                             is_small_preprocess=True),
#                     train_indices=dataset_properties['indices'],
#                     val_indices=testdataset_properties['indices']
#                     )
#              )
    
# Fit the pipeline
pipeline.fit(X=dict(X_train=train_dataset.dataset ,
             dataset_properties=dict(
                 mean=mean.numpy(),
                 std=std.numpy(),
                 num_classes=4,
                 num_features=dataset[0][0].shape[0] * dataset[0][0].shape[1],
                 image_height=dataset[0][0].shape[1],
                 image_width=dataset[0][0].shape[2],
             ),
             train_indices=dataset_properties['indices'],
             val_indices=testdataset_properties['indices']
             )
)
print(pipeline)

Pipeline CS:
 ________________________________________ 
Configuration space object:
  Hyperparameters:
    image_augmenter:GaussianBlur:sigma_min, Type: UniformFloat, Range: [0.0, 3.0], Default: 0.0
    image_augmenter:GaussianBlur:sigma_offset, Type: UniformFloat, Range: [0.0, 3.0], Default: 0.5
    image_augmenter:GaussianBlur:use_augmenter, Type: Categorical, Choices: {True, False}, Default: True
    image_augmenter:GaussianNoise:sigma_offset, Type: UniformFloat, Range: [0.0, 3.0], Default: 0.3
    image_augmenter:GaussianNoise:use_augmenter, Type: Categorical, Choices: {True, False}, Default: True
    image_augmenter:RandomAffine:rotate, Type: UniformInteger, Range: [0, 360], Default: 45
    image_augmenter:RandomAffine:scale_offset, Type: UniformFloat, Range: [0.0, 0.4], Default: 0.2
    image_augmenter:RandomAffine:shear, Type: UniformInteger, Range: [0, 45], Default: 30
    image_augmenter:RandomAffine:translate_percent_offset, Type: UniformFloat, Range: [0.0, 0.4], Default: 0.2

TypeError: Expected X_train to be instance of (<class 'numpy.ndarray'>, <class 'pandas.core.frame.DataFrame'>, <class 'scipy.sparse._base.spmatrix'>) got <class 'torchvision.datasets.folder.ImageFolder'>